In [1]:
import os
import time
import re
import json
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [2]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1920x1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

### category

In [3]:
category = ["Chinos",
"Gilets",
"Jackets & Coats",
"Jeans",
"Knitwear",
"Long Sleeve Lounge Tops",
"Long Sleeve Polos",
"Long Sleeve Tees",
"Lounge Tops",
"Polos",
"Shirt Jackets",
"Shirts",
"Short Sleeve Shirts",
"Short Sleeve Sweats",
"Shorts",
"Sweat Pants",
"Sweats",
"T-Shirts",
"Swimwear",
"Track Tops",
"Trousers"]

### 아이템 태그 리스트 만들기

In [5]:
url = makeUrl("T-Shirts", 3)
driver.get(url)
all_items = driver.find_elements_by_css_selector("a.sc-1koxpgo-0.bTJixI.sc-5sgtnq-2.gHSLMJ")

In [6]:
items = []
for a in all_items:
    item_url = a.get_attribute("href")     #세부 아이템 url
#     name에 brand가 같이 있음  >> 추후 전처리 과정 필요
    item_name = a.find_element_by_css_selector("span.sc-5sgtnq-3.dMAnEc").text
    item_color = a.find_element_by_css_selector("span.sc-5sgtnq-4.dMAnEd").text
    items.append({"name":item_name, "href":item_url, "color":item_color})

In [7]:
items

[{'name': 'Comme des Garcons Play Printed Heart Logo Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-printed-heart-logo-tee-p1t180-1.html',
  'color': 'Navy'},
 {'name': 'Comme des Garcons Play Text Logo Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-text-logo-tee-p1t176-1.html',
  'color': 'Navy'},
 {'name': 'Comme des Garcons Play Double Heart Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-double-heart-tee-p1t226-1.html',
  'color': 'Black'},
 {'name': 'Comme des Garcons Play Double Heart Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-double-heart-tee-p1t226-3.html',
  'color': 'Grey'},
 {'name': 'Comme des Garcons Play Double Heart Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-double-heart-tee-p1t226-2.html',
  'color': 'Navy'},
 {'name': 'Comme des Garcons Play Red Heart Logo Tee',
  'href': 'https://www.endclothing.com/kr/comme-des-garcons-play-red-heart-logo-tee

### 세부 데이터 크롤링

In [8]:
item_detail_url = items[4]["href"]
item_detail_url

'https://www.endclothing.com/kr/comme-des-garcons-play-double-heart-tee-p1t226-2.html'

In [9]:
driver.get(item_detail_url)

In [10]:
# 데이터 저장 dict
detail_item_info = {}

# 접속
driver.get(item_detail_url)

# 기본정보
detail_item_info["shop"] = "endclothing"
detail_item_info["brand"] = driver.find_elements_by_css_selector("a.sc-1koxpgo-0.bTJixI.sc-1exjb4p-1.cpyBme")[2].find_element_by_css_selector("span").text
cp_brand = detail_item_info["brand"].upper()
detail_item_info["product"] = driver.find_element_by_css_selector("span.sc-1klajxh-2.FkpyQ").text.upper().strip(cp_brand).strip(" ")
# detail_item_info["product_id"] = driver.find_element_by_css_selector("").text

# 이미지url
imgs = driver.find_element_by_css_selector("div.ribuxi-0.hxzMGr").find_elements_by_css_selector("img")
img_list = []
for img in imgs:
    detail_item_img = img.get_attribute("src")
    img_list.append(detail_item_img)
detail_item_info["img"] = img_list

# 상세정보
# infos = driver.find_element_by_css_selector("div.sc-1q7zf5z-1.jWPxIw").find_elements_by_css_selector("li")
# for row in infos:
#     row.text

In [11]:
detail_item_info

{'shop': 'endclothing',
 'brand': 'Comme des Garçons Play',
 'product': 'UBLE HEART T',
 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playdoublehearttee_navy_p1t226-2_jd_1.jpg',
  'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playdoublehearttee_navy_p1t226-2_jd_2.jpg',
  'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playdoublehearttee_navy_p1t226-2_jd_m1.jpg',
  'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playdoublehearttee_navy_p1t226-2_jd_m2.jpg',
  'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playdoublehearttee_navy_p1t226-2_jd_m3.jpg']}

## 개별 함수

### make url

In [4]:
def makeUrl(category, page):
    url = f"https://www.endclothing.com/kr/clothing?department={category}&page={page}"
    return url

### 카테고리, 페이지 기반 전체 아이템 크롤러


In [14]:
def getItemList(category, page):
    url = makeUrl(category, page)
    driver.get(url)
    all_items = driver.find_elements_by_css_selector("a.sc-1koxpgo-0.bTJixI.sc-5sgtnq-2.gHSLMJ")
    items = []
    for a in all_items:
        item_url = a.get_attribute("href")     #세부 아이템 url
    #     name에 brand가 같이 있음  >> 추후 전처리 과정 필요
        item_name = a.find_element_by_css_selector("span.sc-5sgtnq-3.dMAnEc").text
        item_color = a.find_element_by_css_selector("span.sc-5sgtnq-4.dMAnEd").text
        items.append({"name":item_name, "href":item_url, "color":item_color})
        
    return items

In [15]:
getItemList("Polos", 2)

[{'name': 'Fred Perry x Raf Simons Printed Patch Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-printed-patch-polo-sm8128-608.html',
  'color': 'Navy'},
 {'name': 'Fred Perry x Raf Simons Printed Patch Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-printed-patch-polo-sm8128-102.html',
  'color': 'Black'},
 {'name': 'Fred Perry x Raf Simons Slim Fit Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-slim-fit-polo-sm8121-102.html',
  'color': 'Black'},
 {'name': 'Fred Perry x Raf Simons Slim Fit Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-slim-fit-polo-sm8121-608.html',
  'color': 'Navy'},
 {'name': 'Fred Perry x Raf Simons Laurel Detail Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-laurel-detail-polo-sm8122-102.html',
  'color': 'Black'},
 {'name': 'Fred Perry x Raf Simons Laurel Detail Polo',
  'href': 'https://www.endclothing.com/kr/fred-perry-x-raf-simons-

### 세부 아이템 페이지 크롤러

In [6]:
def crawl_item(item_detail_url):
    # 데이터 저장 dict
    detail_item_info = {}

    # 접속
    driver.get(item_detail_url)

    # 기본정보
    detail_item_info["shop"] = "endclothing"
    detail_item_info["brand"] = driver.find_elements_by_css_selector("a.sc-1koxpgo-0.bTJixI.sc-1exjb4p-1.cpyBme")[2].find_element_by_css_selector("span").text
    cp_brand = detail_item_info["brand"].upper()
    detail_item_info["product"] = driver.find_element_by_css_selector("span.sc-1klajxh-2.FkpyQ").text.upper().strip(cp_brand).strip(" ")
    # detail_item_info["product_id"] = driver.find_element_by_css_selector("").text

    # 이미지url
    imgs = driver.find_element_by_css_selector("div.ribuxi-0.hxzMGr").find_elements_by_css_selector("img")
    img_list = []
    for img in imgs:
        detail_item_img = img.get_attribute("src")
        img_list.append(detail_item_img)
    detail_item_info["img"] = img_list

    # 상세정보
    # infos = driver.find_element_by_css_selector("div.sc-1q7zf5z-1.jWPxIw").find_elements_by_css_selector("li")
    # for row in infos:
    #     row.text
    return detail_item_info

### 꾸공용 item id

In [7]:
def make_id(num,shop="00", category="00"):
#     10 : 무신사
#     11 : 29cm
#     12 : endclothing
#     13 : farfetch
#     14 : matchesfashion
#     15 : oco
    item_id = shop + category + str(num).rjust(5, "0")
    return item_id

In [37]:
# 아이템 dict을 저장할 dict
item_group = dict()
num = 1
for item in items:
    item_id = make_id(num, "12", "01")
    item_group[item_id] = crawl_item(item["href"])
    print(item_group[item_id])
    num += 1
    
item_group

{'shop': 'endclothing', 'brand': 'Wood Wood', 'product': 'PCL TEE', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-01-2020_woodwood_pcltee_black_12015718-2334-bk_jb_m3.jpg']}
{'shop':

{'shop': 'endclothing', 'brand': 'Comme des Garçons Play', 'product': 'ITTLE RED HEART T', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredhearttee_white_p1t200-2_blr_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredhearttee_white_p1t200-2_blr_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredhearttee_white_p1t200-2_blr_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredhearttee_white_p1t200-2_blr_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredhearttee_white_p1t200-2_blr_m3.jpg']}
{'shop': 'endclothing', 'brand': 'Comme des Garçons Play', 'pr

KeyboardInterrupt: 

In [16]:
def crawl_category(sub_category, page, path=None, save=False):
    # 아이템 dict을 저장할 dict
    item_group = dict()
    num = 1

    # page 별
    for p in range(1, int(page + 1)):
        # 페이지 item리스트 뽑기
        itemList = getItemList(sub_category, p)
        print("page :", p)
        print(itemList)
        # 개별 아이템 json화
        for item in itemList:
            item_id = make_id(num, "12", "00")
            a = crawl_item(item["href"])
            a["_id"] = item_id
            a["color"] = item["color"]
            a["href"] = item["href"]
            item_group[item_id] = a
            print(a)
            num += 1

    # json으로 저장
    if save == True:
        # path 설정
        if path == None:
            path = f"./itemList.json"
        else:
            path = path + "/itemList.json"
        with open(path, "w", encoding="utf-8") as make_file:
            json.dump(item_group, make_file, ensure_ascii=False, indent="\t")

    return item_group

In [17]:
crawl_category("Polos", 2, save=True)

page : 1
[{'name': 'Human Made Polo', 'href': 'https://www.endclothing.com/kr/human-made-polo-hm19cs029-blu.html', 'color': 'Blue'}, {'name': 'Barbour Beacon Polo', 'href': 'https://www.endclothing.com/kr/barbour-beacon-polo-mml0961ny31.html', 'color': 'New Navy'}, {'name': 'Barbour Beacon Polo', 'href': 'https://www.endclothing.com/kr/barbour-beacon-polo-mml0961wh11.html', 'color': 'White'}, {'name': 'Barbour Beacon Polo', 'href': 'https://www.endclothing.com/kr/barbour-beacon-polo-mml0961bl57.html', 'color': 'Pale Sky'}, {'name': 'Barbour Beacon Polo', 'href': 'https://www.endclothing.com/kr/barbour-beacon-polo-mml0961pi34.html', 'color': 'Burnt Coral'}, {'name': 'McQ Alexander McQueen Tipped Swallow Patch Polo', 'href': 'https://www.endclothing.com/kr/mcq-alexander-mcqueen-tipped-swallow-patch-polo-277624rmt72-9000.html', 'color': 'Optic White'}, {'name': 'Maison Kitsuné Double Fox Head Patch Polo', 'href': 'https://www.endclothing.com/kr/maison-kitsune-double-fox-head-patch-polo-bu

{'shop': 'endclothing', 'brand': 'Human Made', 'product': 'POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_m1_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_2_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_3_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_4_1.jpg'], '_id': '120000001', 'color': 'Blue', 'href': 'https://www.endclothing.com/kr/human-made-polo-hm19cs029-blu.html'}
{'shop': 'endclothing', 'brand': 'Barbour', 'product': 'BEACON POL', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/B/a/Barbour-Beacon-Polo---New-Navy---_MML0961NY31_m1_1.jpg', 'https://media.endclothing.com/media/

{'shop': 'endclothing', 'brand': 'Alexander McQueen', 'product': 'GON PATCH POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/9/09-12-2019_alexandermcqueen_dragonpatchpolo_black_609601qox33-1000_mb_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/9/09-12-2019_alexandermcqueen_dragonpatchpolo_black_609601qox33-1000_mb_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/9/09-12-2019_alexandermcqueen_dragonpatchpolo_black_609601qox33-1000_mb_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/9/09-12-2019_alexandermcqueen_dragonpatchpolo_black_609601qox33-1000_mb_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/9/09-12-2019_alexandermcqueen_dragonpatchpolo_black_609601qox33-1000_mb_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmed

{'shop': 'endclothing', 'brand': 'Fred Perry Authentic', 'product': 'WIN TIPPED KNITTED POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/F/r/Fred-Perry-Authentic-Twin-Tipped-Knitted-Polo-Snow-White-_K8500-129_m1_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/F/r/Fred-Perry-Authentic-Twin-Tipped-Knitted-Polo-Snow-White-_K8500-129_2_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/F/r/Fred-Perry-Authentic-Twin-Tipped-Knitted-Polo-Snow-White-_K8500-129_3_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/F/r/Fred-Perry-Authentic-Twin-Tipped-Knitted-Polo-Snow-White-_K8500-129_5_1.jpg'], '_id': '120000017', 'color': 'Snow White', 'href': 'https://www.endclothing.com/kr/fred-perry-authentic-twin-tipped-knitted-polo-k8500-129.html'}
{'shop': 'endclothing', 'brand': 'Fred Perry Authentic', 'product': 'MULT

{'shop': 'endclothing', 'brand': 'Comme des Garçons Play', 'product': 'ITTLE RED HEART', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredheartpolo_navy_p1t204-2_jb_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredheartpolo_navy_p1t204-2_jb_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredheartpolo_navy_p1t204-2_jb_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredheartpolo_navy_p1t204-2_jb_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/2/7/27-02-2020_commedesgarcons_playlittleredheartpolo_navy_p1t204-2_jb_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/

...

{'shop': 'endclothing', 'brand': 'Fred Perry Laurel Wreath', 'product': 'ISSUES ORIGINAL TWIN TIPPED POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/3/03-02-2020_fredperry_reissuesoriginaltwintippedpolo_greymarl_lavender_m12-k28_jb_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/3/03-02-2020_fredperry_reissuesoriginaltwintippedpolo_greymarl_lavender_m12-k28_jb_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/3/03-02-2020_fredperry_reissuesoriginaltwintippedpolo_greymarl_lavender_m12-k28_jb_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/3/03-02-2020_fredperry_reissuesoriginaltwintippedpolo_greymarl_lavender_m12-k28_jb_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/0/3/03-02-2020_fredperry_reissuesoriginaltwintippedpolo_greymarl_lavender_m1

{'shop': 'endclothing', 'brand': 'Fred Perry Authentic', 'product': 'MADE IN JAPAN POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_fredperry_authenticmadeinjapanpolo_black_m102-253_blr_1x.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_fredperry_authenticmadeinjapanpolo_black_m102-253_blr_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_fredperry_authenticmadeinjapanpolo_black_m102-253_blr_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_fredperry_authenticmadeinjapanpolo_black_m102-253_blr_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_fredperry_authenticmadeinjapanpolo_black_m102-253_blr_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/

{'shop': 'endclothing', 'brand': 'Sunspel', 'product': 'RIVIERA POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026-gyab_ml_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026-gyab_ml_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026-gyab_ml_3.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026-gyab_ml_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026-gyab_ml_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/3/0/30-01-2020_sunspel_rivierapolo_charcoal_mpol1026

{'shop': 'endclothing', 'brand': 'Saint Laurent', 'product': 'D COLLAR POLO', 'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-12-2019_saintlaurent_studcollarpolo_greymarl_607493-yblg2-1463_aj_1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-12-2019_saintlaurent_studcollarpolo_greymarl_607493-yblg2-1463_aj_2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-12-2019_saintlaurent_studcollarpolo_greymarl_607493-yblg2-1463_aj_m1.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-12-2019_saintlaurent_studcollarpolo_greymarl_607493-yblg2-1463_aj_m2.jpg', 'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/1/6/16-12-2019_saintlaurent_studcollarpolo_greymarl_607493-yblg2-1463_aj_m3.jpg'], '_id': '120000234', 'color': 'Grey Marl', 'href': 'https://www.endclot

{'120000001': {'shop': 'endclothing',
  'brand': 'Human Made',
  'product': 'POLO',
  'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_m1_1.jpg',
   'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_2_1.jpg',
   'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_3_1.jpg',
   'https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/H/u/Human-Made-Polo-Blue-_HM19CS029-BLU_4_1.jpg'],
  '_id': '120000001',
  'color': 'Blue',
  'href': 'https://www.endclothing.com/kr/human-made-polo-hm19cs029-blu.html'},
 '120000002': {'shop': 'endclothing',
  'brand': 'Barbour',
  'product': 'BEACON POL',
  'img': ['https://media.endclothing.com/media/f_auto,q_auto:eco/prodmedia/media/catalog/product/B/a/Barbour-Beacon-Polo---New-Navy---_MML0